<a href="https://colab.research.google.com/github/Pushkar-Bhuse/Political-Influence/blob/master/SentimentAnalysis_(SVC_TF_IDF).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':"1gJcmE0W1uFiHuhuivCjE1MA2hTZJuXNW"})   # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv')        # replace the file name with your file

In [0]:
downloaded = drive.CreateFile({'id':"1y3FJe_RN9m8kBK5jthChmRez21F5gx5H"})   # replace the id with id of file you want to access
downloaded.GetContentFile('test.csv')  

In [0]:
downloaded = drive.CreateFile({'id':"1nsByWtC9AdsT7-rVmxBWcE0ziwYkQzqc"})   # replace the id with id of file you want to access
downloaded.GetContentFile('stop_hinglish.txt') 

In [0]:
downloaded = drive.CreateFile({'id':"1r72zfm_HmyVSHmRmV7zR0ZWpp9zj_sHu"})   # replace the id with id of file you want to access
downloaded.GetContentFile('BJP_dataset.csv') 

In [0]:
downloaded = drive.CreateFile({'id':"1TxurggTc0vJNSQoGeP6o4nnqrfV2al-U"})   # replace the id with id of file you want to access
downloaded.GetContentFile('CONGRESS_dataset.csv') 

In [0]:
drive = GoogleDrive(gauth)
your_module = drive.CreateFile({"id": "1PTjUlqfdULPkXLoM1Xw7jhXzkuIHQybu"})   # "your_module_file_id" is the part after "id=" in the shareable link
your_module.GetContentFile("hinglish_stemmer.py")          # Save the .py module file to Colab VM
import hinglish_stemmer

In [0]:
downloaded = drive.CreateFile({'id':"18LaDPTiQH4j1onJQRvfS6rrZINInrbpF"})   # replace the id with id of file you want to access
downloaded.GetContentFile('w2vModel') 

In [11]:
train_data = pd.read_csv("train.csv", encoding='ISO-8859-1')
train_data.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [12]:
bjp_data = pd.read_csv("BJP_dataset.csv")
bjp_data = bjp_data.rename(columns = {'commentText': 'SentimentText', 'Label': 'Sentiment'})

congress_data = pd.read_csv("CONGRESS_dataset.csv")
congress_data = congress_data.rename(columns = {'commentText': 'SentimentText', 'Label': 'Sentiment'})

bjp_data.head()

,SentimentText,Sentiment
0,Banda apna 100 % best he,1
1,WE NEED THE KING MAKER ---- NARENDRA MODI,1
2,Awesome... Banda apna Best hai,1
3,Best PM ever,1
4,RAHUL GANDHI LEFT THE CHAT,1


In [0]:
bjp_data['about_bjp'] = 1
bjp_data['about_congress'] = 0

congress_data['about_bjp'] = 0
congress_data['about_congress'] = 1

train_data = pd.concat([bjp_data, congress_data], ignore_index = True)

In [14]:
train_data.tail()

,SentimentText,Sentiment,about_bjp,about_congress
3945,modi ji is best,0,0,1
3946,vote for modi,0,0,1
3947,Who is the watching this video after air strik...,0,0,1
3948,Chai wala haven't done shit time to throw him ...,0,0,1
3949,"These people only seek their profit,dipshits",0,0,1


In [15]:
 #Checking Tweets
rand_indexs = np.random.randint(1,len(train_data),10).tolist()
train_data["SentimentText"][rand_indexs]

535                                                best h
2269                                        Modi is thief
605                                             Only modi
683                                 don’t forget congress
1314                                        only Congress
3209                                            nice song
2143              Modi is the best...the best...\nRahuluu
3890    BJP will win all the seats . Vote for BJP. Mod...
1760                                         go back modi
488                     Vote for BJP ðŸš©ðŸ‡®ðŸ‡³ðŸ‡®ðŸ‡³
Name: SentimentText, dtype: object

In [16]:
tweets_text = train_data.SentimentText.str.cat()
emos = set(re.findall(r" ([xX:;][-']?.) ",tweets_text))
emos_count = []
for emo in emos:
    emos_count.append((tweets_text.count(emo), emo))
sorted(emos_count,reverse=True)[:10]

[(2, ':D')]

In [17]:
HAPPY_EMO = r" ([xX;:]-?[dD)]|:-?[\)]|[;:][pP]) "
SAD_EMO = r" (:'?[/|\(]) "
print("Happy emoticons:", set(re.findall(HAPPY_EMO, tweets_text)))
print("Sad emoticons:", set(re.findall(SAD_EMO, tweets_text)))

Happy emoticons: {':D'}
Sad emoticons: set()


In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
def most_used_words(text):
    tokens = word_tokenize(text)
    frequency_dist = nltk.FreqDist(tokens)
    print("There is %d different words" % len(set(tokens)))
    return sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)

In [0]:
# most_used_words(train_data.SentimentText.str.cat())[:10]

In [21]:
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [0]:
from nltk.corpus import words

fobj = open("stop_hinglish.txt")
stopwords = fobj.read().split()

def tokenization(text):
  #Removing Stopwords (hinglish + english)
  new_text = []
  for token in word_tokenize(text):
    if token in stopwords:
      continue
    else:
      new_text.append(token)

  #Stemming both hinglish and english words
  
  emmatizer = WordNetLemmatizer()
  hinglishStemmer = hinglish_stemmer.Stemmer()

  final_text = []
  for token in new_text:
      if token in words.words():
        final_text.append(lemmatizer.lemmatize(token))
      else:
        final_text.append(hinglishStemmer.stemWord(token))
  
  return final_text


In [23]:
train_data['SentimentText'] = train_data['SentimentText'].str.replace("#", "")
train_data['SentimentText'] = train_data['SentimentText'].str.replace(r"[-\.\n]", "")
# Removing HTML garbage
train_data['SentimentText'] = train_data['SentimentText'].str.replace(r"&\w+;", "")
# Removing links
train_data['SentimentText'] = train_data['SentimentText'].str.replace(r"https?://\S*", "")
# replace repeated letters with only two occurences
# heeeelllloooo => heelloo
train_data['SentimentText'] = train_data['SentimentText'].str.replace(r"(.)\1+", r"\1\1")
# mark emoticons as happy or sad
train_data['SentimentText'] = train_data['SentimentText'].str.replace(HAPPY_EMO, " happyemoticons ")
train_data['SentimentText'] = train_data['SentimentText'].str.replace(SAD_EMO, " sademoticons ")
train_data['SentimentText'] = train_data['SentimentText'].str.lower()

train_data.head(10)

,SentimentText,Sentiment,about_bjp,about_congress
0,banda apna 100 % best he,1,1,0
1,we need the king maker narendra modi,1,1,0
2,awesome banda apna best hai,1,1,0
3,best pm ever,1,1,0
4,rahul gandhi left the chat,1,1,0
5,what congressi will make?? is thr any quality ...,1,1,0
6,bjp zindabaad,1,1,0
7,vote for modi jii vande matram,1,1,0
8,modi brand of india youth like him so much mr...,1,1,0
9,superb bhai,1,1,0


In [0]:
#Handling tags, In this case we remove all the tags
train_data['SentimentText'] = train_data['SentimentText'].str.replace(r"@[a-zA-Z0-9_]* ", "")

In [63]:
sentiments = train_data['Sentiment']
tweets = train_data['SentimentText']

#using TF-IDF Vectorizer
vectorizer = TfidfVectorizer(tokenizer = tokenization)

clf = SVC(probability=True)

grid_search_pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('SVC', clf)
])

params = [
    {
        'vectorizer__max_features': [1000, 2000, 5000, 10000, 20000, None],
        'vectorizer__ngram_range': [(1,2), (1,3)],
        'SVC__C': [0.1, 1, 10, 100, 1000],  
        'SVC__gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
        'SVC__kernel': ['rbf', 'linear', 'poly']
    },
]

grid_search = GridSearchCV(grid_search_pipeline, params, cv=5, scoring='f1')
grid_search.fit(tweets, sentiments)
print(grid_search.best_params_)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
LookupError: 
**********************************************************************
  Resource words not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('words')
  
  Searched in:
    - '/root/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/usr/nltk_data'
    - '/usr/lib/nltk_data'

LookupError: ignored

In [0]:
#nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True